In [ ]:
import os
import torch
import torch.nn as nn
import numpy as np
from torch.nn.utils import clip_grad_norm_
# from data_utils import Dicionary, Corpus

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
class Dictionary(object):
    def __init__(self):
        self.word2idx = {}
        self.idx2word = {}
        self.idx = 0
    
    def add_word(self, word):
        if not word in self.word2idx:
            self.word2idx[word] = self.idx
            self.idx2word[self.idx] = word
            self.idx += 1
    
    def __len__(self):
        return len(self.word2idx)

In [ ]:
class Corpus(object):
    def __init__(self):
        self.dictionary = Dictionary()
    
    def get_data(self, path, batch_size=32):
        with open(path, 'r') as f:
            tokens = 0
            for line in f:
                words = line.split() + ['<eos>']
                tokens += len(words)
                for word in words:
                    self.dictionary.add_word(word)
        
        ids = torch.LongTensor(tokens)
        token = 0
        with open(path, 'r') as f:
            for line in f:
                words = line.split() + ['<eos>']
                for word in words:
                    ids[token] = self.dictionary.word2idx[word]
                    token += 1
        
        num_batches = ids.size(0) // batch_size
        ids = ids[:num_batches*batch_size]
        return ids.view(batch_size, -1)

In [ ]:
data_root = './data'
sample_dir = './tutorial-09-samples'

if not os.path.exists(sample_dir):
    os.makedirs(sample_dir)

embed_size = 129
hidden_size = 1024
num_layers = 1
num_epochs = 5
num_samples = 1000  # number of words to be sampled
batch_size = 32
seq_length = 30
learning_rate = 0.002

In [ ]:
corpus = Corpus()
ids = corpus.get_data(os.path.join(data_root, 'train.txt'), batch_size)
vocab_size = len(corpus.dictionary)
num_batches = ids.size(1) // seq_length

In [ ]:
class RNNLM(nn.Module):
    def __init__(self, vocab_size, embed_size, hidden_size, num_layers):
        super(RNNLM, self).__init__()
        self.embed = nn.Embedding(vocab_size, embed_size)
        self.lstm = nn.LSTM(embed_size, hidden_size, num_layers, batch_first=True)
        self.linear = nn.Linear(hidden_size, vocab_size)
    
    def forward(self, x, h):
        x = self.embed(x)

        out, (h, c) = self.lstm(x, h)

        out = out.reshape(out.size(0)*out.size(1), out.size(2))

        out = self.linear(out)
        return out, (h, c)

In [ ]:
model = RNNLM(vocab_size, embed_size, hidden_size, num_layers).to(device)
print(model)

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
def detach(states):
    return [state.detach() for state in states]

In [ ]:
for epoch in range(num_epochs):
    states = (torch.zeros(num_layers, batch_size, hidden_size).to(device),
              torch.zeros(num_layers, batch_size, hidden_size).to(device))
    
    for i in range(0, ids.size(1)-seq_length, seq_length):
        inputs = ids[:, i:i+seq_length].to(device)
        targets = ids[:, (i+1):(i+1)+seq_length].to(device)

        states = detach(states)
        outputs, states = model(inputs, states)
        
        loss = criterion(outputs, targets.reshape(-1))
        optimizer.zero_grad()
        loss.backward()
        clip_grad_norm_(model.parameters(), 0.5)
        optimizer.step()

        step = (i+1)//seq_length
        if step%100 == 0:
            print ('Epoch [{}/{}], Step[{}/{}], Loss: {:.4f}, Perplexity: {:5.2f}'
                   .format(epoch+1, num_epochs, step, num_batches, loss.item(), np.exp(loss.item())))

In [ ]:
with torch.no_grad():
    with open(os.path.join(sample_dir, 'sample.txt'), 'w') as f:
        state = (torch.zeros(num_layers, 1, hidden_size).to(device),
                 torch.zeros(num_layers, 1, hidden_size).to(device))
        
        prob = torch.ones(vocab_size)
        input = torch.multinomial(prob, num_samples=1).unsqueeze(1).to(device)

        for i in range(num_samples):
            output, state = model(input, state)
            
            prob = output.exp()
            word_id = torch.multinomial(prob, num_samples=1).item()

            input.fill_(word_id)

            word = corpus.dictionary.idx2word[word_id]
            word = '\n' if word == '<eos>' else word + ' '
            f.write(word)

            if (i+1)%100 == 0:
                print('Sampled [{}/{}] words and save to {}'.format(i+1, num_samples, os.path.join(sample_dir, 'sample.txt')))

In [ ]:
torch.save(model.state_dict(), os.path.join(sample_dir, 'model.ckpt'))